In [17]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [18]:
# Load the crypto_data.csv dataset.
df = pd.read_csv("Resources/crypto_data.csv",index_col=0)
len(df)
df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Index,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000


In [19]:
# Keep all the cryptocurrencies that are being traded.
tradingcryptos = df[df["IsTrading"] == True]
len(tradingcryptos)

1144

In [20]:
# Remove the "IsTrading" column.
removetrading = tradingcryptos.drop(["IsTrading"], axis = 1)
removetrading.shape

(1144, 5)

In [21]:
# Remove rows that have at least 1 null value.
nona = removetrading.dropna()
nona.shape

(685, 5)

In [22]:
# Keep the rows where coins are mined.
crypto_df = nona[nona["TotalCoinsMined"] != 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Index,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [23]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name = pd.DataFrame(crypto_df["CoinName"])
crypto_name.head(10)

,CoinName
Index,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
LTC,Litecoin
DASH,Dash
XMR,Monero
ETC,Ethereum Classic


In [24]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop("CoinName", axis= 1)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Index,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [25]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns = ["Algorithm","ProofType"])
X = X.drop("CoinName", axis= 1)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Index,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# Standardize the data with StandardScaler().
le = LabelEncoder()
crypto_scaled = StandardScaler().fit_transform(X)
print(crypto_scaled)

[[-0.11674788 -0.15286468 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.09358885 -0.14499604 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [ 0.52587231  4.4937636  -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 ...
 [-0.09523411 -0.13215444 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.11658774 -0.15255408 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.11674507 -0.15284989 -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [27]:
# Using PCA to reduce dimension to three principal components.
# Initialize PCA model
pca = PCA(n_components=3)
# Get two principal components for the iris data.
crypto_pca = pca.fit_transform(crypto_scaled)
print(crypto_pca)

[[-0.33824655  1.1581585  -0.55288295]
 [-0.32156847  1.15822372 -0.55329781]
 [ 2.30921153  1.73292464 -0.63789313]
 ...
 [ 0.32622359 -2.31624985  0.42033623]
 [-0.15810872 -2.12168518  0.36538218]
 [-0.30033312  0.67938299 -0.30370702]]


In [28]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index = crypto_name.index)
pcs_df.head(10)

,PC 1,PC 2,PC 3
Index,,,
42,-0.338247,1.158159,-0.552883
404,-0.321568,1.158224,-0.553298
1337,2.309212,1.732925,-0.637893
BTC,-0.137559,-1.354812,0.178554
ETH,-0.155007,-2.039824,0.330441
LTC,-0.170286,-1.073065,0.002986
DASH,-0.380633,1.354995,-0.506964
XMR,-0.144097,-2.198865,0.396292
ETC,-0.153448,-2.039928,0.330428


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [29]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# Create the Elbow Curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [35]:
# Initialize the K-Means model with k = 4.
model = KMeans(n_clusters=4, random_state=0)
model

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [58]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df,pcs_df],axis =1)
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = crypto_name['CoinName']
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"]= model.labels_
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class
Index,,,,,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.338247,1.158159,-0.552883,0
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.321568,1.158224,-0.553298,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.309212,1.732925,-0.637893,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.137559,-1.354812,0.178554,1
ETH,Ethereum,Ethash,PoW,1.076842e+08,0,-0.155007,-2.039824,0.330441,1
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000,-0.170286,-1.073065,0.002986,1
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000,-0.380633,1.354995,-0.506964,0
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0,-0.144097,-2.198865,0.396292,1
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000,-0.153448,-2.039928,0.330428,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [63]:
# Creating a 3D-Scatter with the PCA data and the clusters
figure = px.scatter_3d(
    clustered_df,
    hover_name="CoinName",
    hover_data=["Algorithm"],
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class"
)
figure.update_layout(legend=dict(x=0,y=1))
figure.show()


In [66]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(sortable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class]

In [69]:
# Print the total number of tradable cryptocurrencies.
print(len(clustered_df))

533


In [70]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
print(scaled)

[[4.20000000e-11 5.94230127e-03]
 [5.32000000e-04 7.00182308e-03]
 [3.14159265e-01 3.53420682e-02]
 ...
 [1.40022261e-03 6.92655266e-03]
 [2.10000000e-05 5.94962775e-03]
 [1.00000000e-06 5.94243008e-03]]


In [75]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_clustered_df = pd.DataFrame(scaled, columns = ["TotalCoinSupply", "TotalCoinsMined"], index = clustered_df.index)
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_clustered_df = pd.concat([scaled_clustered_df,clustered_df["CoinName"]],axis =1)
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_clustered_df = pd.concat([scaled_clustered_df,clustered_df["Class"]], axis=1)
scaled_clustered_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
Index,,,,
42,4.200000e-11,0.005942,42 Coin,0
404,5.320000e-04,0.007002,404Coin,0
1337,3.141593e-01,0.035342,EliteCoin,0
BTC,2.100000e-05,0.005960,Bitcoin,1
ETH,0.000000e+00,0.006050,Ethereum,1
LTC,8.400000e-05,0.006006,Litecoin,1
DASH,2.200000e-05,0.005951,Dash,0
XMR,0.000000e+00,0.005960,Monero,1
ETC,2.100000e-04,0.006056,Ethereum Classic,1


In [76]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_clustered_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols = ["CoinName"])


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)